In [7]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# 各種数値

In [8]:
grav = 9.81

In [9]:
# 順に、軸荷重、径荷重、軸振動数、径振動数
rocket = np.array([6.0, 2.5, 30.0, 10.0])
reentry = np.array([80.0, 0.0, 0.0, 0.0])

#要求を決定
required = np.max([rocket,reentry],axis=0)

In [18]:
torayca_t800h = {'density':1.6, #g/cm^3
                 'tension':2850, #MPa
                 'comp':1700, #MPa
                 'E':155, #GPa
                 'poisson':0.3  
                }

kevlar49 = {'density':1.45, #g/cm^3
            'tension':3000, #MPa
            'comp':300, #MPa
            'E':112.4, #GPa
            'poisson':0.3  
            }

al7075 = {'density':2.8, #g/cm^3
          'tension':550, #MPa
          'comp':500, #MPa
          'E':70, #GPa
          'shear':330, #MPa
          'poisson':0.3  
         }

al_5052_0002 = {'density':0.091, #g/cm^3
          'E':1.97, #GPa
          'F_sy':0.784, #MPa
          'shear_1':550, #MPa
          'shear_2':200 #MPa
         }

In [19]:
def get_k_c(material,L,r,t):
    Z_L = L**2/r/t * np.sqrt(1-material['poisson']**2)
    k_c = (10**(-0.6))*Z_L
    return k_c

In [20]:
def get_sigma_buckling_cylinder(material,L,r,t):
    k_c = get_k_c(material,L,r,t)
    E = material['E'] *1e9
    nu = material['poisson']
    return k_c*np.pi**2*E/12/((1-nu)**2)*(t/L)**2

In [21]:
def get_sigma(required,L,r,t,mass):
    """
    最終的にこのsigmaがsigma_buckling_cylinderを下回っていることを確認したい
    """
    F = required[0] * mass * grav
    A = 2*np.pi*r*t
    return F/A

In [22]:
def get_f_axis(material,M,L,r,t):
    A = 2*np.pi*r*t
    E = material['E'] *1e9
    f_axis = 0.250*np.sqrt(A*E/M/L)
    return f_axis

def get_f_rad(material,M,L,r,t):
    E = material['E']*1e9
    I = np.pi*(L**4-(L-t)**4)/64
    f_rad = 0.560*np.sqrt(E*I/M/L**3)
    return f_rad

In [23]:
def check(material,L,r,t,mass):
    log = ['不合格','合格']
    
    sigma_buckling_cylinder = get_sigma_buckling_cylinder(material,L,r,t)
    sigma = get_sigma(required,L,r,t,mass)
    print('=================================')
    print('縦荷重:{0}'.format(sigma))
    print('縦荷重許容値:{0}'.format(sigma_buckling_cylinder))
    print('縦荷重合否:{0}'.format(log[sigma_buckling_cylinder>sigma]))
    print('縦荷重安全率:{0}\n'.format(sigma_buckling_cylinder/sigma))
    
    print('=================================')    
    f_axis = get_f_axis(material,mass,L,r,t)
    f_axis_req = required[2]
    print('縦振動数:{0}'.format(f_axis))
    print('縦振動数最低値:{0}'.format(f_axis_req))
    print('縦振動数合否:{0}'.format(log[f_axis>f_axis_req]))
    print('縦振動数安全率:{0}\n'.format(f_axis/f_axis_req))
    
    print('=================================')    
    f_rad = get_f_rad(material,mass,L,r,t)
    f_rad_req = required[3]
    print('横振動数:{0}'.format(f_rad))
    print('横振動数最低値:{0}'.format(f_rad_req))
    print('横振動数合否:{0}'.format(log[f_rad>f_rad_req]))
    print('横振動数安全率:{0}\n'.format(f_rad/f_rad_req))

In [45]:
class Honeycomb_cylinder():
    def __init__(self,
                 facesheet_material,
                 core_material,
                 h_c,
                 t_f,
                 L,r,t
                ):
        self.facesheet = facesheet_material
        self.core = core_material
        self.h_c = h_c #[m]
        self.t_f = t_f #[m]
        self.L = L
        self.r = r
        self.t = t
        
        E = self.facesheet['E'] * 1e9
        nu = self.facesheet['poisson']
        
        self.D1 = E*self.t_f*(self.h_c+self.t_f)**2/2/(1-nu)**2
        self.Df = E * self.t_f**3/12/(1-nu)**2
        self.D = self.D1 + 2*self.Df
        
        G_c = self.core['shear_1'] * 1e6
        self.U = self.h_c * G_c
        
    def get_P_cr_1(self):
        P_1 = np.pi**2 * self.D1/(4*self.r**2)
        P_2 = 2*np.pi**2 * self.Df/(4*self.r**2)
        P_cr_1 = P_1 * (P_2 + self.U*(1+P_2/P_1))/(P_1+self.U)
        return P_cr_1
    
    def get_sigma_cr_1(self):
        P_1 = np.pi**2 * self.D1/(4*self.r**2)
        P_2 = 2*np.pi**2 * self.Df/(4*self.r**2)
        P_cr_1 = P_1 * (P_2 + self.U*(1+P_2/P_1))/(P_1+self.U)
        return P_cr_1/2/self.t_f
    
    def get_sigma_cr_2(self):
        V = np.pi**2*self.D/(2*r)**2/self.U
        beta = 1
        m = np.arange(0,10,1)
        K_c_list = np.pi**2*self.D/(2*r)**2*((m/beta + beta/m)**2/(1+V * (1+m**2/beta**2)))
        K_c = np.min(K_c_list)
        
        P_cr_2 = K_c * np.pi**2 * self.D/(2*r)**2
        return P_cr_2/2/self.t_f
    
    def get_sigma_cr_3(self):
        E_f = self.facesheet['E']*1e9
        E_c = self.core['E']*1e9
        G_c = self.core['shear_1'] * 1e6
        sigma_cr_3 = 0.5*(E_f*E_c*G_c)**(1/3)
        return sigma_cr_3
        
    def get_sigma_cr(self):
        E = self.facesheet['E'] * 1e9
        nu = self.facesheet['poisson']
        G_c = self.core['shear_1'] * 1e6
        
        term1 = (self.h_c + self.t_f)/self.r
        term2 = E / np.sqrt(1-nu**2)
        term3 = (1 - self.h_c*self.t_f/2/self.r/(self.h_c+self.t_f) * (E/G_c)/np.sqrt(1-nu**2) )
        
        return term1*term2*term3
    
    def get_mass(self,shape):
        d_f = self.facesheet['density'] * 1e3 #[kg/m^3]
        d_c = self.core['density'] * 1e3
        
        if shape=='cylinder':
            Area = 2*np.pi*self.r * self.L
        elif shape=='circle':
            Area = np.pi*self.r**2
        Density_per_area = d_f*self.t_f + d_c*self.h_c
        return Area * Density_per_area
    
    def get_freq(self,shape):
        d_f = self.facesheet['density'] * 1e3 #[kg/m^3]
        d_c = self.core['density'] * 1e3
        mu = d_f*self.t_f + d_c*self.h_c
        return 4.98/self.r**2*np.sqrt(self.D/mu)/2/np.pi
    
    
    
    def check(self,shape,mass,acc=[6,1.5]):
        if shape == 'cylinder':
            # 圧縮座屈
            # 質量
            None
        elif shape == 'circle':
            P = mass * grav * acc[0] / 2 / self.r
            
            # 表皮材曲げ
            sigma_f_max = P*2*self.r/4/self.h_c/self.t_f
            sigma_f = self.facesheet['tension'] * 1e6
            print('最大表皮板応力:{0}'.format(sigma_f_max))
            print('引っ張り強さ:{0}'.format(sigma_f))
            print('安全余裕:{0}\n'.format(sigma_f/sigma_f_max - 1))
            
            # コア材剪断
            tau_c_max = P/(self.t_f+self.h_c)
            tau_c = self.core['F_sy'] * 1e6
            print('最大コア材剪断応力:{0}'.format(tau_c_max))
            print('剪断強さ:{0}'.format(tau_c))
            print('安全余裕:{0}\n'.format(tau_c/tau_c_max - 1))            
            
            # 面内圧縮によるEuler座屈
            P = mass * grav * acc[1] / 2 / self.r
            P_cr_1 = self.get_P_cr_1()
            print('面内Euler座屈:{0}'.format(P_cr_1))
            print('圧縮荷重:{0}'.format(P))
            print('安全余裕:{0}\n'.format(P_cr_1/P - 1))             
            
        return 0

In [46]:
r1 = 0.5
L2 = 1.2

r3 = 1.0
L4 = 1.2

L6 = 1.0

h_c = 0.020
t_f = 0.001
hc = Honeycomb_cylinder(facesheet_material = al7075,
                 core_material = al_5052_0002,
                 h_c = h_c,
                 t_f = t_f,
                 L=L2,r=r3,t=t_f+h_c,
                )
hc.get_mass('cylinder')

34.83397934300363

In [47]:
hc.check(shape='circle',mass=100)

最大表皮板応力:73575000.0
引っ張り強さ:550000000.0
安全余裕:6.475365273530411

最大コア材剪断応力:140142.85714285713
剪断強さ:784000.0
安全余裕:4.594291539245668

面内Euler座屈:77236.56395746964
圧縮荷重:735.75
安全余裕:103.97664146445075



0

In [34]:
hc.get_freq('circle')

65.47082808454425

In [376]:
print(hc.get_sigma_cr_1())
print(hc.get_sigma_cr_2())
print(hc.get_sigma_cr_3())

22465767.83620777
62569543828693.68
2116471003.744568


/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in true_divide


In [ ]:
np.pi/(2*r1)**2*np.sqrt()

In [40]:
hc.get_sigma_cr()

1100652443.701975

In [139]:
get_sigma([6],L4,r3,t_f+h_c,mass=2861) * 1.5

3654742.816099476

In [263]:
#節点3
mass_1 = 961
h_c = 0.01
t_f = 0.001

P_f = mass_1 * grav * 1.8
sigma_f = P_f / (2*r3*(t_f)) *1.5

tau_c = P_f / (2*t_f)  * 1.5

In [264]:
tau_c

12727003.5

In [187]:
#試しに1段目について
material = kevlar49
L = 0.6
r = 0.5
t = 0.01
mass = 100

check(material,L,r,t,mass)

縦荷重:2498095.986770389
縦荷重許容値:904147784.3278147
縦荷重合否:合格
縦荷重安全率:361.9347651635769

縦振動数:1917.8846155538267
縦振動数最低値:30.0
縦振動数合否:合格
縦振動数安全率:63.929487185127556

横振動数:821.5803324796384
横振動数最低値:10.0
横振動数合否:合格
横振動数安全率:82.15803324796384



/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  if __name__ == '__main__':
/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
